BOOTH'S ALGORITHM

Take the input

In [32]:
# def get_input():
#     m = int(input("Enter the multiplicand: "))
#     q = int(input("Enter the multiplier: "))
#     return m,q
# m, q = get_input()
m = -9 # Multiplicand
q = 7  # Multiplier
n = 5  # Number of bits

In [33]:
def to_binary(num):
  if(num >= 0):
    return(bin(num)[2:].zfill(n)) #[2:] - to remove 0b
  elif(num < 0):
    return (bin(abs(num))[2:].zfill(n))
# print(to_binary(9))
# print(to_binary(-2))

In [34]:
def add(x1, x2, n):
  result = ''
  carry = 0
  for i in range(n - 1, -1, -1):
      carry += 1 if x1[i] == '1' else 0
      carry += 1 if x2[i] == '1' else 0
      result = ('1' if carry % 2 == 1 else '0') + result
      carry = 0 if carry < 2 else 1
  return(result.zfill(n))
# add('0010','1010',4)

In [35]:
def twos_comp(num,n):
  x = ''
  one_add = '0'*(n-1)+'1' # 0001
  one_c = list(map(lambda x: '0' if x=='1' else '1',num)) # 0111 --> 1000
  # print(one_c)
  for i in one_c:
    x += i 
  two_c = add(x,one_add,n)
  return two_c
print(twos_comp('10101',n))

01011


In [36]:
def ashr(bits):
    msb = bits[0]
    shift_bits = bin(int(('0b' + bits),2) >> 1)[2:]
    if(msb == '0'):
        bits = shift_bits.zfill(2*n+1)
    else:
        bits = msb + shift_bits
    return bits   
# print(ashr('001110011'))
# print(ashr('101110011'))

In [37]:
def table(bits,count,oper,n):
    A = bits[0:n]
    q_bin = bits[n:2*n]
    q_minus_1 = bits[2*n]
    print(f'  {count}     {A}   {q_bin}      {q_minus_1}      {oper}')

BOOTH'S ALGORITHM

![](BoothAlgo.jpg)

In [38]:
def Booth_Algo(bits,count,n):
    A = bits[:n]
    q_bin = bits[n:2*n]
    q_minus_1 = bits[2*n]
    if(bits[-2] == '1' and bits[-1] == '0'):
        A = add(A,minus_M,n)
        bits = A + q_bin + q_minus_1
        table(bits,count,'A = A - M',n)
        bits = ashr(bits)
        count -= 1
        table(bits,count,'Shift Right',n)
    elif(bits[-2] == '0' and bits[-1] == '1'):
        A = add(A,plus_M,n)
        bits = A + q_bin + q_minus_1
        table(bits,count,'A = A + M',n)
        bits = ashr(bits)
        count -= 1
        table(bits,count,'Shift Right',n)
    elif(bits[-2] == '0' and bits[-1] == '0'):
        bits = ashr(bits)
        count -= 1
        table(bits,count,'Shift Right',n)
    elif(bits[-2] == '1' and bits[-1] == '1'):
        bits = ashr(bits)
        count -= 1
        table(bits,count,'Shift Right',n)
    if(count != 0):
        Booth_Algo(bits,count,n)
    else:
        A = bits[0:n]
        q_bin = bits[n:2*n]
        print(f'A : {A} , q : {q_bin}')
        x = A + q_bin
        if((m>0 and q>0) or (m<0 and q<0)):
            result = int(('0b' + x),2)
            print(f"Product of {m} and {q} is {result}")        
        elif(m<0 or q<0):
            result = int(('0b' + twos_comp(x,2*n)),2)
            print(f"Product of {m} and {q} is {-result}")

In [39]:
if(m>=0 and q>=0):
    plus_M = to_binary(m)
    minus_M = twos_comp(plus_M,n)
    q_bin = to_binary(q)
elif(m<=0 and q>=0):
    plus_M = twos_comp(to_binary(m),n)
    minus_M = to_binary(m)
    q_bin = to_binary(q)   
elif(m>=0 and q<=0):
    plus_M = to_binary(m)
    minus_M = twos_comp(plus_M,n)
    q_bin = twos_comp(to_binary(q),n)
elif(m<=0 and q<=0):
    plus_M = twos_comp(to_binary(m),n)
    minus_M = to_binary(m)
    q_bin = twos_comp(to_binary(q),n)  

q_minus_1 = '0'
A = '0'*n
count=n

# Trace the table
print('count  ','  A  ','    q  ','q_minus_1','  Operation')

# INITIALISATION
bits = A + q_bin + q_minus_1
table(bits,count,'Initialisation',n)
Booth_Algo(bits,count,n)

count     A       q   q_minus_1   Operation
  5     00000   00111      0      Initialisation
  5     01001   00111      0      A = A - M
  4     00100   10011      1      Shift Right
  3     00010   01001      1      Shift Right
  2     00001   00100      1      Shift Right
  2     11000   00100      1      A = A + M
  1     11100   00010      0      Shift Right
  0     11110   00001      0      Shift Right
A : 11110 , q : 00001
Product of -9 and 7 is -63
